In [1]:
import pandas as pd
from pathlib import Path
import numpy as np
import os

In [2]:
contaminants_data = []

contaminants_list = ["2,3,7,8-Tetrachlorodibenzo-p-dioxin","Dieldrin","Hexachlorobiphenyl; 3,3',4,4',5,5'- (PCB 169)","Pentachlorobiphenyl; 3,3',4,4',5- (PCB 126)","Mercury","Lead","Cyanide"]
clean_data_dir = '../cleandata'

for filename in os.listdir(clean_data_dir):
    if filename.endswith(".csv"):
        csv_data = pd.read_csv(filename, parse_dates=True, infer_datetime_format=True)
        for item in contaminants_list:
            contaminants_filtered = csv_data[csv_data['CHEMICAL_NAME'] == item]
            contaminants_filtered = contaminants_filtered.iloc[: , 1:]
            contaminants_data.append(contaminants_filtered)

contaminants_df = pd.concat(contaminants_data)
contaminants_df

,LOC_NAME,SAMPLE_DATE,TASK_CODE,ANALYTIC_METHOD,CAS_RN,CHEMICAL_NAME,REPORT_RESULT_VALUE,REPORT_RESULT_UNIT,REPORT_RESULT_LIMIT,DETECT_FLAG,REPORTABLE_RESULT,LONGITUDE,LATITUDE
0,Hackensack River,7/23/1985,1981-2014 RPI(Bopp) SED/ SUS Matter,E1613B MLA017,1746-01-6,"2,3,7,8-Tetrachlorodibenzo-p-dioxin",48.3,pg/g,0.485,Y,Yes,-74.039502,40.879778
244,Hackensack River,7/30/1985,1981-2014 RPI(Bopp) SED/ SUS Matter,E1613B MLA017,1746-01-6,"2,3,7,8-Tetrachlorodibenzo-p-dioxin",229.0,pg/g,0.481,Y,Yes,-74.069464,40.798054
488,Kill Van Kull,8/20/1981,1981-2014 RPI(Bopp) SED/ SUS Matter,E1613B MLA017,1746-01-6,"2,3,7,8-Tetrachlorodibenzo-p-dioxin",55.4,pg/g,0.495,Y,Yes,-74.098822,40.648703
732,Newark Bay,8/21/1985,1981-2014 RPI(Bopp) SED/ SUS Matter,E1613B MLA017,1746-01-6,"2,3,7,8-Tetrachlorodibenzo-p-dioxin",108.0,pg/g,0.481,Y,Yes,-74.139111,40.671200
976,Newark Bay,4/26/1985,1981-2014 RPI(Bopp) SED/ SUS Matter,E1613B MLA017,1746-01-6,"2,3,7,8-Tetrachlorodibenzo-p-dioxin",113.0,pg/sample,0.671,Y,Yes,-74.139111,40.671200
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15876,LPR-0403-01,2019-07-09 11:00:00,2019 OU2 PDI Porewater Passive Sampler,SW6010,7439-92-1,Lead,400.0,mg/kg,0.990,Y,Yes,-74.145593,40.735957
15881,LPR-0430-07,2019-07-10 10:45:00,2019 OU2 PDI Porewater Passive Sampler,SW6010,7439-92-1,Lead,370.0,mg/kg,1.100,Y,Yes,-74.149512,40.734412
15884,LPR-0752-01R,2019-07-09 16:10:00,2019 OU2 PDI Porewater Passive Sampler,SW6010,7439-92-1,Lead,11.0,mg/kg,0.730,Y,Yes,-74.155944,40.770182
15889,LPR-0790-03R,2019-07-09 14:15:00,2019 OU2 PDI Porewater Passive Sampler,SW6010,7439-92-1,Lead,51.0,mg/kg,0.640,Y,Yes,-74.152242,40.774933


In [3]:

contaminants_df.to_csv('../../notebooks/chemical_filter.csv')







In [49]:
molar_mass={
        "2,3,7,8-Tetrachlorodibenzo-p-dioxin":321.97,
        "Cyanide":26.02,
        "Dieldrin":380.91,
        "Hexachlorobiphenyl; 3,3',4,4',5,5'- (PCB 169)":360.878,
        "Lead":207.2,
        "Mercury":200.59,
        "Pentachlorobiphenyl; 3,3',4,4',5- (PCB 126)":326.433
}

conversion_factor={
        'pg/g':10**-6,
        'pg/sample':10**-6,
        'ppt':10**-6,
        'ppb':10**-3,
        'ppm':1,
        'ug/kg':10**-3,
        'UMOLES/G':1,
        'ng/g':10**-3,
        'umol/g':1,
        'mg/kg':1,
        'ng/kg':10**-6,
        'pg':10**-4,
        'pg/l':10**-9,
        'ng/l':10**-6,
        'ug/l':10**-3,
        'mg/l':1
}
contaminants_df['REPORT_RESULT_UNIT'].fillna('ug/kg',inplace=True)
contaminants_df.dropna(subset=['REPORT_RESULT_VALUE'],inplace=True)
contaminants_df.isnull().sum()

def unit_conversion(row):
    REPORT_RESULT_VALUE, REPORT_RESULT_UNIT = row.REPORT_RESULT_VALUE, row.REPORT_RESULT_UNIT
    conversion = conversion_factor[REPORT_RESULT_UNIT]
    
    if REPORT_RESULT_UNIT == 'UMOLES/G' or REPORT_RESULT_UNIT == 'umol/g':
        return REPORT_RESULT_VALUE**2
    else:
    
        return REPORT_RESULT_VALUE*conversion


contaminants_df['SCALED_VALUE'] = contaminants_df.apply(unit_conversion,axis=1)

def value_moles(row):
        CHEMICAL_NAME, SCALED_VALUE = row.CHEMICAL_NAME, row.SCALED_VALUE
        molar = molar_mass[CHEMICAL_NAME]

        return SCALED_VALUE / molar

contaminants_df['VALUE_MUMOL_PER_GRAM'] = contaminants_df.apply(value_moles,axis=1)

contaminants_df.sample(20)


,LOC_NAME,SAMPLE_DATE,TASK_CODE,ANALYTIC_METHOD,CAS_RN,CHEMICAL_NAME,REPORT_RESULT_VALUE,REPORT_RESULT_UNIT,REPORT_RESULT_LIMIT,DETECT_FLAG,REPORTABLE_RESULT,LONGITUDE,LATITUDE,value_ug/g,VALUE_UG/G,SCALED_VALUE,VALUE_MUMOL_PER_GRAM
22732,PR9901SDM,1999-10-11 12:00:00,1999 Late Summer-Early Fall ESP Sampling,MET,7439-92-1,Lead,271000.00000,ug/kg,NaN,Y,Yes,-74.119928,40.729510,2.710000e+02,2.710000e+02,2.710000e+02,1.307915e+00
44911,71A,12/1/1992,1992 Core Sediment Investigation,E335.2,57-12-5,Cyanide,672.00000,ug/kg,672.00000,N,Yes,-74.151324,40.659931,6.720000e-01,6.720000e-01,6.720000e-01,2.582629e-02
7967,16A,12/2/1991,1991 Core Sediment Investigation,PCB/PEST,60-57-1,Dieldrin,3.59000,ug/kg,3.59000,N,Yes,-74.139845,40.740002,3.590000e-03,3.590000e-03,3.590000e-03,9.424799e-06
4924,PR0010SDM,2000-05-18 12:00:00,2000 Spring ESP Sampling Program,PCB CONGENER,57465-28-8,"Pentachlorobiphenyl; 3,3',4,4',5- (PCB 126)",2.52000,ug/kg,2.52000,N,Yes,-74.166290,40.748609,2.520000e-03,2.520000e-03,2.520000e-03,7.719808e-06
46365,Arcadis Treatability Study Lab,2019-06-04 15:45:00,2018-2019 OU2 PDI Treatability Study,E1699,60-57-1,Dieldrin,780000.00000,pg/g,1380.00000,Y,Yes,-74.144568,40.735829,7.800000e-01,7.800000e-01,7.800000e-01,2.047728e-03
43243,PR9909CRAB,1999-10-22 12:00:00,1999 Late Summer-Early Fall ESP Sampling,PCB CONGENER,57465-28-8,"Pentachlorobiphenyl; 3,3',4,4',5- (PCB 126)",0.10600,ug/kg,NaN,Y,Yes,-74.164005,40.739029,1.060000e-04,1.060000e-04,1.060000e-04,3.247221e-07
11499,PR9905SDL,1999-10-10 12:00:00,1999 Late Summer-Early Fall ESP Sampling,PCB CONGENER,32774-16-6,"Hexachlorobiphenyl; 3,3',4,4',5,5'- (PCB 169)",0.45500,ug/kg,0.45500,N,Yes,-74.136977,40.742111,4.550000e-04,4.550000e-04,4.550000e-04,1.260814e-06
1071,RM 7.8,2018-11-19 11:45:00,2018-2019 OU2 PDI Treatability Study,SW6010,7439-92-1,Lead,350.00000,mg/kg,0.61000,Y,Yes,-74.152960,40.774340,3.500000e+02,3.500000e+02,3.500000e+02,1.689189e+00
13926,LPR-0765-01,2019-05-28 11:00:00,2019 OU2 PDI Porewater Passive Sampler,E1668A,57465-28-8,"Pentachlorobiphenyl; 3,3',4,4',5- (PCB 126)",69.20000,pg/g,69.20000,N,Yes,-74.155138,40.771860,6.920000e-05,6.920000e-05,6.920000e-05,2.119884e-07
2425,WTH-7,2/1/1995,1995 USACE Minish Park Investigation,MET,7439-92-1,Lead,700000.00000,ug/kg,NaN,Y,Yes,-74.164179,40.739403,7.000000e+02,7.000000e+02,7.000000e+02,3.378378e+00


In [5]:

# create a funciton that takes chemical_df 'REPORT_RESULT_VALUE' and 
# multiplies it by value in unit_conversion key of chemical_conversion_dict 
# that corresponds to 'REPORT_RESULT_UNIT' for that row and finally divides by molar_mass key in 
# chemical_conversion_dict that corrsponds to 'CHEMICAL_NAME'

def chemical_conversion(row):
    CHEMICAL_NAME, REPORT_RESULT_VALUE, REPORT_RESULT_UNIT = row.CHEMICAL_NAME, row.REPORT_RESULT_VALUE, row.REPORT_RESULT_UNIT
    conversion = conversion_factor[REPORT_RESULT_UNIT]
    molar = molar_mass[contaminants_df[6]]
    amount = contaminants_df[REPORT_RESULT_VALUE]
    return (amount*conversion)/molar

contaminants_df['SAMPLE_AMOUNT_CONVERTED'] = contaminants_df.apply(chemical_conversion, axis=1)


KeyError: 6